In [13]:
import numpy as np
import math
import random

# Get input:
This function gets an integer, then get the requests and stock length for that case.

In [14]:
def open_file(i):
    # Open input file
    if (i == 1):
        infile = open('input1.stock', 'r')
    elif (i == 2):
        infile = open('input2.stock', 'r')
    elif (i == 3):
        infile = open('input3.stock', 'r')
    else:
        infile = open('input4.stock', 'r')

    # Read instance header for the pr1002
    length = int(infile.readline().strip().split()[2]) #length
    infile.readline()
    infile.readline()
    requests = np.array(list(map(int, infile.readline().strip().split(', ')[:-1]))) #requests

    # Close input file
    infile.close()
    
    return length, requests

# Cost function:
This function calculates the cost of the given order. The given order, determines cuts one by one.

This is how to calculate cost:
1. Until all the requests have been read:
2. Add the request to last stock.
3. If the sum of requests got more than stock length, subtract the request from sum.
4. Add one to used stock number.

In [15]:
def calculate_cost(order, length):
    cost = 0
    s = 0
    i = 0
    while(s <= length and i < len(order)):
        # Add the request
        s += order[i]
        if (s > length):
            # Subtract the request and add one to number of stocks
            s -= order[i]
            i -= 1
            cost += 1
            s = 0
        i += 1
    return cost

# Neighbour:
Using this function, we determine the neighbour(next order) of current order, considering the temperature.
This is how we do it:
1. First we get two random numbers for index.
2. Then we change the order, swapping the random indexes.
3. If we get the lower cost by this new order, we accept the new order.
4. Otherwise, by chance of: $$\frac{\exp^{(cost - new\_cost)}}{temperature}$$ we accept the new order.

In [16]:
def neighbour(order, temperature, length):
    new_order = order.copy()
    # random numbers
    i = random.randint(0, len(order) - 1)
    j = random.randint(0, len(order) - 1)
    
    # new order
    temp = new_order[i]
    new_order[i] = new_order[j]
    new_order[j] = temp
    
    # Calculate costs
    cost = calculate_cost(order, length)
    new_cost = calculate_cost(new_order, length)
    
    # IF the new order was better
    if (new_cost < cost):
        order = new_order
        return order
        
    # If the new order was not better. accept it by some chance
    rand = random.random()
    if (rand < math.exp((cost - new_cost)/temperature)):
        order = new_order
    
    return order

# Main function:
Given temperature, alpha, TL and the data:
1. We get the data.
2. Create a permutation of requests.
3. Then, while temperature > 0.1:
4. TL times, do:
5. Find a neibour to this order.
6. If the order is better than best order, update the best order.
7. update the temperature by: $$temperature = temperature*\alpha$$

In [17]:
def simulated_annealing(temperature, alpha, TL, i):
    # Get the input
    length, requests = open_file(i)
    
    # Create a permutation
    requests = np.random.permutation(requests)

    best_order = []
    least_cost = np.Inf
    costs = []
    best_costs = []
    while(temperature > 0.1):
        for i in range(TL):
            # Find the neighbour
            requests = neighbour(requests, temperature, length)
            
            # If the cost was lower than best order, update the least cost
            if (calculate_cost(requests, length) < least_cost):
                best_order = requests
                least_cost = calculate_cost(requests, length)
                
            costs.append(calculate_cost(requests, length))
            best_costs.append(least_cost)
        temperature *= alpha
        
    return best_order, least_cost, costs, best_costs

results of input1, which finds an order, using only $\bf{51}$ stock to cover the requests.

The order given, determines that in each step which request must be handled.

In [18]:
order, least, costs, best_costs = simulated_annealing(1, 0.99, 1000, 1) #input = input1.stock
print(order)
print(least)

[672 266 346 354  53 678 109 280 660 117 506 356 515 145 116  23 315 549
 148 460 337 988 186  61 457  70 187 463 119 405 115 753 437 284 632 149
  84 224 753 441 241 312 581 301 249 689  75 532 286  80 501 412 170  18
 107 592  46  60  88 351 248 868 557 306  92 518 135 149 292 627 144 557
 266 933 409 517 507 149 283 232  69 371 106 123  92 211 686  79 365 230
 402 967 180 788 805 118 125 557 295 648 126 218 251  33 495 286 653 788
 106 987 716 268 264 424  88  99 125 662 312 370  45 525 181 246 555 914
 368  43 544 278 609 106 106 618  71 414 171 333  78]
51


results of input2, which finds an order, using only $\bf{78}$ stock to cover the requests.

The order given, determines that in each step which request must be handled.

In [211]:
order, least, costs, best_costs = simulated_annealing(1, 0.99, 1000, 2) #input = input2.stock
print(order)
print(least)

[2100 2200 1380 2100 2050 2200 2200 2150 1380 1710 1820 1820 1380 1820
 1820 1880 2140 1380 2000 1820 1820 1710 1710 1380 2050 1520 2200 1520
 1880 1710 1710 1820 1560 2140 1380 2200 1820 2140 2000 1380 2050 1560
 2100 2200 2000 2050 1520 1710 1820 1520 2140 2150 1380 1820 2050 1930
 1520 2140 2200 2140 1930 1560 1560 1880 1380 1930 2100 1930 1520 1710
 1930 1820 1560 1880 2100 1560 2100 1880 1820 1880 1930 1520 2140 1380
 2200 1710 2150 2150 2000 2050 1520 2150 2200 1560 2150 1880 2140 1380
 1880 1520 2200 1560 2100 1820 1560 2200 1930 1710 2050 1380 2150 1520
 1520 2150 1930 1520 2050 1520 2000 1520 2150 1880 1930 2200 1880 2200
 1380 1930 1520 2100 1380 2100 2100 1380 1930 2150 2050 1380 2100 1560
 1820 2140 1380 2150 1820 2000 2000 1520 2000 2150 1380 1880 1560 1930
 2150 2200 2100 1710 1520 1820 2050 1710 1880 1520 2140 1880 2200 1520
 2050 1930 1520 2200 2000 2140 1380 1930 1380 1930 2140 1710 1880 1520
 2140 1820 1930 2150 1520 1560 2100 1880 1820 2000 2150 2200 1710 1880
 1880 

results of input3, which finds an order, using only $\bf{96}$ stock to cover the requests.

The order given, determines that in each step which request must be handled.

In [214]:
order, least, costs, best_costs = simulated_annealing(1, 0.99, 1000, 3) #input = input3.stock
print(order)
print(least)

[  7   8  52  99 255  49   2   5  10  11  29 403   2   3   5 161   1   1
 313   5 421 405   4 280   3  14   6   3  10   8 174   8 229   3  91   5
   1  18   7  21  87  85   4   8   1   4   6 359   6   4 152   1   7  12
 277   3  16  16  50 159 275  18   1  22 144   4  23 264   2  13   9 251
   2   7   2 225  18   6 116   2  14   4  13   1 167  37  90   1 147  41
   1 134   2 169   5 110  12  17   1 152 134  11  10   9   2   6  11   2
 335   1 152   1 172  13   1   9 170   6   2 102   5   8  11   6   2   2
  19   3  92   3   7   9   2  10   5   3 321  14   1   8 108 318   3   4
  10   2  10   5   2   8  21   4   1   3  43   3 153   1  18 225   8   1
   5   1   5   9 271 133  16   3  26  88   2   6 245   6   1   2  17   7
   6  14   9  27   4   7  10   4 204 239  26 196  16   1 279   1   2   1
   3   4   9  45  40 250   5   7   7   4  10  93  78  12   9  14   4   8
   8   3   4 152   2 199   3 263   1   7  16 162   2   7   3   8   9 386
   1   1   1  96   3   9  11   9   2   2   9   7 17

results of input4, which finds an order, using only $\bf{218}$ stock to cover the requests.

The order given, determines that in each step which request must be handled.

In [213]:
order, least, costs, best_costs = simulated_annealing(1, 0.99, 1000, 4) #input = input4.stock
print(order)
print(least)

[ 1 45  6 27  5  6 36 37 22 80 10  9 32 16 52 22 29  4 30 12  9 43  6 10
  2  6 21 17 58  6 13 31 51 15  3 32 46 20 13  3  9 73 37 11 20 28  2  4
 24 68 54 17  8  6  1 80 18 41 19 29  4  4 10  5 16  9 44 23  1 23  2 20
  5 20 44 26 11 15 10 38  6 29 14 59  6 20  8  7 73 19 46 54  7  8 16 67
 20 16  5  2 40  4 20 46  1 25 34  3 32 30  9 53  4 30 30 24 25 28 12  5
  2 28 24 21 36  4 17 18 63 35 98 76 10 15 68 15  2 79  7  7 13 13  4 10
 52  5 33  1 10  9 45 39 33  8 12  3 36 18 22 23  7  5 65  4 18 25  8  3
  3  1  8 47 65  1 11 56  2  1 39  2 37 13 14  8 18  8 20  2 47 25 16 24
  6  3  7 38 65 33  5 20  7 14 38  7 15  9 23 44  7 11 14  5  9  9 21 16
 87  1 36 50  3  5  3 12 81 32 41  7  2 21 78  4  2 32  1 30  4  4 14 18
 16  4  6  4  5 36  5  3  1 60  8  5 11  7 21 59  6 67 27 48 34 18 11 68
 13  8  5 27  3 47 11  6 17 43  6  9 22  1 20 13  9 24 34 26 13 33 16 51
 12  2  5  3 14 18 10  4 15  7 22  6 15  3  3  5 16  4 15  4 24 26 20 65
 11 73 23  3  5  4 17  3 15 51 21 13 17  5  3  2 16